# Importing

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from scipy.stats import pointbiserialr, spearmanr
#importing all the required ML packages
from sklearn.linear_model import LogisticRegression #logistic regression
from sklearn import svm #support vector Machine
from sklearn.ensemble import RandomForestClassifier #Random Forest
from sklearn.neighbors import KNeighborsClassifier #KNN
from sklearn.naive_bayes import GaussianNB #Naive bayes
from sklearn.tree import DecisionTreeClassifier #Decision Tree
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split #training and testing data split
from sklearn import metrics #accuracy measure
from sklearn.metrics import confusion_matrix #for confusion matrix
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

# Read the demographics data

In [2]:
# read the file
df = pd.read_csv('demographics.csv')
df.head()

,custid,Name,Job,Age,Lat,Long,Income,CreditScore,HouseholdSize,ZipCode,MajorCity,State,MedianHomeValue,MedianHouseholdIncome,Debt
0,92886957,Barbara Thompson,Automotive Service Technicians and Mechanics,39,31.84568,-102.36764,43415,658,4,79761,Odessa,TX,94700,44174,23878.25
1,16462625,James Wade,Prosthodontists,93,42.73920,-84.62081,564164,786,4,48917,Lansing,MI,138000,51830,5641.64
2,89457436,Joel Jackson,Surgeons,99,44.27804,-88.27205,466344,676,1,54140,Little Chute,WI,139200,58247,233172.00
3,68462543,Tyler Clements,Physicians and Surgeons,100,33.50921,-111.89903,756171,694,2,85250,Scottsdale,AZ,250200,59358,378085.50
4,63315493,Adam Stephens,"Railroad Brake, Signal, and Switch Operators",97,33.93113,-117.54866,58948,688,2,92860,Norco,CA,385100,84905,30063.48


# Overview

In [3]:
# overview before cleaning
# axis=0 -> generate for each column
df_before = pd.DataFrame(index=df.columns)
df_before['Datatypes'] = df.dtypes
df_before['Unique'] = df.nunique()
df_before['Max'] = df.max(axis=0)
df_before['Min'] = df.min(axis=0)
df_before['Total'] = df.count(axis=0)
df_before['mean'] = df.median()
df_before['MissingValues'] = df.isnull().any()
df_before['MissingValuesTotal'] = df.isnull().sum()
df_before

,Datatypes,Unique,Max,Min,Total,mean,MissingValues,MissingValuesTotal
custid,int64,48303,99999810,2684,48427,5.003374e+07,False,0
Name,object,39257,Zoe Young,Aaron Adams,48427,NaN,False,0
Job,object,1042,Zoologists and Wildlife Biologists,Accountants and Auditors,48427,NaN,False,0
Age,int64,83,100,18,48427,5.900000e+01,False,0
Lat,float64,156,47.8053,20.8895,48427,3.932011e+01,False,0
Long,float64,156,-69.9653,-156.474,48427,-8.766561e+01,False,0
Income,int64,40138,949942,10000,48427,4.840700e+04,False,0
CreditScore,int64,180,799,620,48427,6.670000e+02,False,0
HouseholdSize,int64,9,10,1,48427,3.000000e+00,False,0
ZipCode,int64,156,99216,1420,48427,4.806000e+04,False,0


In [4]:
df.head(10)

,custid,Name,Job,Age,Lat,Long,Income,CreditScore,HouseholdSize,ZipCode,MajorCity,State,MedianHomeValue,MedianHouseholdIncome,Debt
0,92886957,Barbara Thompson,Automotive Service Technicians and Mechanics,39,31.84568,-102.36764,43415,658,4,79761,Odessa,TX,94700,44174,23878.25
1,16462625,James Wade,Prosthodontists,93,42.73920,-84.62081,564164,786,4,48917,Lansing,MI,138000,51830,5641.64
2,89457436,Joel Jackson,Surgeons,99,44.27804,-88.27205,466344,676,1,54140,Little Chute,WI,139200,58247,233172.00
3,68462543,Tyler Clements,Physicians and Surgeons,100,33.50921,-111.89903,756171,694,2,85250,Scottsdale,AZ,250200,59358,378085.50
4,63315493,Adam Stephens,"Railroad Brake, Signal, and Switch Operators",97,33.93113,-117.54866,58948,688,2,92860,Norco,CA,385100,84905,30063.48
5,58074817,Justin Bryant,Graduate Teaching Assistants,73,28.90054,-81.26367,36360,659,10,32725,Deltona,FL,127300,45059,14180.40
6,57739304,Scott Evans,"Extraction Workers, All Other",67,27.09978,-82.45426,54375,716,9,34285,Venice,FL,153500,45029,9243.75
7,1522679,Carol Sanchez,Chief Executives,20,39.08367,-84.50855,858635,782,4,41014,Covington,KY,88900,30958,103036.20
8,73533078,Stacy Nichols,Emergency Medical Technicians and Paramedics,44,46.09273,-88.64235,37837,659,2,49927,Gaastra,MI,57500,30000,16648.28
9,79412186,Hannah Hall,Family and General Practitioners,68,42.55308,-87.93341,653902,792,4,53142,Kenosha,WI,178600,64930,58851.18
